# IATO Residual Risk Analysis Notebook

This notebook implements **residual risk computation for IATO** using:
- DAG-derived correlation matrix (ρ)
- Beta-Binomial node failures
- Monte Carlo aggregation
- Min/Max/Mean/Std residual risk aggregates
- Trust-object logging

In [ ]:
import sys, os
sys.path.append(os.path.abspath("src"))

from kernel.entropy_pgd import pgd_optimize
from kernel.trust_objects import log_trust_object

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import betabinom

## 1. Define Evidence DAG

In [ ]:
G = nx.DiGraph()
nodes = ['v1','v2','v3','v4']
edges = [('v1','v2'),('v2','v3'),('v1','v4')]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

plt.figure(figsize=(6,4))
nx.draw(G, with_labels=True, node_color='lightblue', node_size=2000, font_size=14, arrowsize=20)
plt.title('Evidence DAG')
plt.show()

## 2. Compute IATO Correlation Matrix (ρ)

In [ ]:
n = len(nodes)
adj_matrix = nx.to_numpy_array(G)

# Base correlation weight for edges
base_corr = 0.3

# Initialize correlation matrix
rho = np.eye(n)  # self-correlation = 1

for i in range(n):
    for j in range(n):
        if i != j and adj_matrix[i,j] == 1:
            rho[i,j] = base_corr * (1 + i/n)  # weighted by node index

print("IATO-derived correlation matrix (ρ):")
print(rho)

## 3. Define Beta-Binomial Node Failure Parameters

In [ ]:
alpha_i = [2,3,2,4]
beta_i  = [5,4,6,3]
N_i     = [10,12,8,15]
n_nodes = len(alpha_i)

## 4. Monte Carlo Residual Risk Computation with Aggregates

In [ ]:
M = 5000  # Monte Carlo draws
R_res_samples = []

for m in range(M):
    # Sample node failures
    f_m = [betabinom.rvs(N_i[j], alpha_i[j], beta_i[j]) for j in range(n_nodes)]
    # Compute residual risk for this draw
    R_m = sum(rho[i,j]*f_m[i] for i in range(n_nodes) for j in range(n_nodes) if i != j)
    R_res_samples.append(R_m)

R_res_samples = np.array(R_res_samples)
R_mean = np.mean(R_res_samples)
R_std  = np.std(R_res_samples)
R_min  = np.min(R_res_samples)
R_max  = np.max(R_res_samples)

print(f"Residual Risk Aggregates:\nMean={R_mean:.2f}, Std={R_std:.2f}, Min={R_min}, Max={R_max}")

## 5. Correlation Matrix Aggregates

In [ ]:
rho_sum   = np.sum(rho) - np.trace(rho)  # sum of off-diagonal correlations
rho_mean  = np.mean(rho[np.where(~np.eye(n,dtype=bool))])
rho_min   = np.min(rho[np.where(~np.eye(n,dtype=bool))])
rho_max   = np.max(rho[np.where(~np.eye(n,dtype=bool))])

print(f"Correlation Matrix Aggregates:\nSum={rho_sum:.2f}, Mean={rho_mean:.2f}, Min={rho_min:.2f}, Max={rho_max:.2f}")

## 6. Log Trust Object

In [ ]:
trust_object = {
    "task": "residual_risk_eval",
    "R_mean": R_mean,
    "R_std": R_std,
    "R_min": R_min,
    "R_max": R_max,
    "rho_sum": rho_sum,
    "rho_mean": rho_mean,
    "rho_min": rho_min,
    "rho_max": rho_max
}
log_trust_object(trust_object)
print("Trust object logged successfully.")

## 7. Residual Risk Distribution Visualization

In [ ]:
plt.hist(R_res_samples, bins=50, color='skyblue', edgecolor='black')
plt.title('Monte Carlo Residual Risk Distribution')
plt.xlabel('Residual Risk')
plt.ylabel('Frequency')
plt.show()